<a href="https://colab.research.google.com/github/danielhou13/cogs402longformer/blob/main/src/Token_attention_with_head_importance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# import sys
# sys.path.append('/content/drive/My Drive/{}'.format("cogs402longformer/"))

In [ ]:
pip install datasets --quiet

In [ ]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Import Dataset and Model

In [ ]:
import os

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

Import the Reserach Papers dataset

In [ ]:
from datasets import load_dataset
from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('allenai/longformer-base-4096')

def longformer_finetuned_papers():
    model = AutoModelForSequenceClassification.from_pretrained('danielhou13/longformer-finetuned_papers', num_labels = 2)
    return model

def preprocess_function(tokenizer, example, max_length):
    example.update(tokenizer(example['text'], padding='max_length', max_length=max_length, truncation=True))
    return example

def get_papers_dataset(dataset_type):
    max_length = 2048
    dataset = load_dataset("danielhou13/cogs402dataset")[dataset_type]

    # tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
    dataset = dataset.map(lambda x: preprocess_function(tokenizer, x, max_length), batched=True)
    setattr(dataset, 'input_columns', ['input_ids', 'attention_mask'])
    setattr(dataset, 'target_columns', ['labels'])
    setattr(dataset, 'max_length', max_length)
    setattr(dataset, 'tokenizer', tokenizer)
    return dataset

def papers_test_set():
    return get_papers_dataset('test')

Import the news dataset

In [ ]:
# def preprocess_function(tokenizer, example, max_length):
#     example.update(tokenizer(example['text'], padding='max_length', max_length=max_length, truncation=True))
#     return example

# def longformer_finetuned_news():
#     model = AutoModelForSequenceClassification.from_pretrained('danielhou13/longformer-finetuned-news-cogs402', num_labels = 2)
#     return model

# def get_news_dataset(dataset_type):
#     max_length = 2048
#     dataset = load_dataset("danielhou13/cogs402dataset2")[dataset_type]

#     tokenizer = AutoTokenizer.from_pretrained('allenai/longformer-base-4096')
#     dataset = dataset.map(lambda x: preprocess_function(tokenizer, x, max_length), batched=True)

#     labels = map(int, dataset['hyperpartisan'])
#     print(type(dataset['hyperpartisan']))
#     labels = list(labels)
#     dataset = dataset.add_column("labels", labels)

#     dataset = dataset.remove_columns(['text', 'title', 'hyperpartisan', 'url', 'published_at', 'bias'])
#     print(dataset)
#     setattr(dataset, 'input_columns', ['input_ids', 'attention_mask'])
#     setattr(dataset, 'target_columns', ['labels'])
#     setattr(dataset, 'max_length', max_length)
#     setattr(dataset, 'tokenizer', tokenizer)
#     return dataset

# def news_train_set():
#     return get_news_dataset('train')

# def news_test_set():
#     return get_news_dataset('validation')

Load papers model and dataset and preprocess it

In [ ]:
cogs402_test = papers_test_set()
model = longformer_finetuned_papers()
columns = cogs402_test.input_columns + cogs402_test.target_columns
print(columns)
cogs402_test.set_format(type='torch', columns=columns)
cogs402_test=cogs402_test.remove_columns(['text'])

Load news model and dataset and preprocess it

In [ ]:
# cogs402_test = news_test_set()
# model = longformer_finetuned_news()
# columns = cogs402_test.input_columns + cogs402_test.target_columns
# print(columns)
# cogs402_test.set_format(type='torch', columns=columns)

In [ ]:
if torch.cuda.is_available():
    model = model.cuda()

print(model.device)

cuda:0


Take example for evaluation

In [ ]:
testexam = cogs402_test[923]

In [ ]:
# print(test['labels'][923])

In [ ]:
output = model(testexam["input_ids"].unsqueeze(0).cuda(), attention_mask=testexam['attention_mask'].unsqueeze(0).cuda(), labels=testexam['labels'].cuda(), output_attentions = True)
batch_attn = output[-2]
output_attentions = torch.stack(batch_attn).cpu()
global_attention = output[-1]
output_global_attentions = torch.stack(global_attention).cpu()
print("output_attention.shape", output_attentions.shape)
print("gl_output_attention.shape", output_global_attentions.shape)

output_attention.shape torch.Size([12, 1, 12, 2048, 514])
gl_output_attention.shape torch.Size([12, 1, 12, 2048, 1])


In [ ]:
# print(os.getcwd())
# yes = torch.load("resources/longformer_test2/epoch_3/aggregate_attn.pt")

Convert sliding attention matrix to correct seq_len x seq_len matrix

In [ ]:
def create_head_matrix(output_attentions, global_attentions):
    new_attention_matrix = torch.zeros((output_attentions.shape[0], 
                                      output_attentions.shape[0]))
    for i in range(output_attentions.shape[0]):
        test_non_zeroes = torch.nonzero(output_attentions[i]).squeeze()
        test2 = output_attentions[i][test_non_zeroes[1:]]
        new_attention_matrix_indices = test_non_zeroes[1:]-257 + i
        new_attention_matrix[i][new_attention_matrix_indices] = test2
        new_attention_matrix[i][0] = output_attentions[i][0]
        new_attention_matrix[0] = global_attentions.squeeze()[:output_attentions.shape[0]]
    return new_attention_matrix


def attentions_all_heads(output_attentions, global_attentions):
    new_matrix = []
    for i in range(output_attentions.shape[0]):
        matrix = create_head_matrix(output_attentions[i], global_attentions[i])
        new_matrix.append(matrix)
    return torch.stack(new_matrix)

def all_batches(output_attentions, global_attentions):
    new_matrix = []
    for i in range(output_attentions.shape[0]):
        matrix = attentions_all_heads(output_attentions[i], global_attentions[i])
        new_matrix.append(matrix)
    return torch.stack(new_matrix)

def all_layers(output_attentions, global_attentions):
    new_matrix = []
    for i in range(output_attentions.shape[0]):
        matrix = all_batches(output_attentions[i], global_attentions[i])
        new_matrix.append(matrix)
    return torch.stack(new_matrix)

In [ ]:
converted_mat = all_layers(output_attentions, output_global_attentions).detach().cpu().numpy()
print(converted_mat.shape)

(12, 1, 12, 2048, 2048)


Sum over all the tokens (column-wise)

In [ ]:
attention_sum = converted_mat.sum(axis=3)
print(attention_sum.shape)

(12, 1, 12, 2048)


Load head importance model and scale the attentions by head importance

In [ ]:
head_importance = torch.load("/content/drive/MyDrive/cogs402longformer/t3-visapplication/resources/pretrained/head_importance.pt")

In [ ]:
print(head_importance[2])

[0.00404374 0.02163236 0.00531127 0.01472499 0.03868961 0.00646301
 0.01300004 0.00334545 1.         0.00796655 0.1583804  0.01080081]


In [ ]:
def scale_by_importance(attention_matrix, head_importance):
  new_matrix = np.zeros_like(attention_matrix)
  for i in range(attention_matrix.shape[0]):
    head_importance_layer = head_importance[i]
    for j in range(attention_matrix.shape[1]):
      print(attention_matrix[i][j].shape)
      new_matrix[i][j] = attention_matrix[i][j] * np.expand_dims(head_importance_layer, axis=1)
  return new_matrix

In [ ]:
attention_matrix_importance = scale_by_importance(attention_sum, head_importance)
print(attention_matrix_importance[:, 0, :, :].shape)

(12, 2048)
(12, 2048)
(12, 2048)
(12, 2048)
(12, 2048)
(12, 2048)
(12, 2048)
(12, 2048)
(12, 2048)
(12, 2048)
(12, 2048)
(12, 2048)
(12, 12, 2048)


In [ ]:
print(attention_matrix_importance)

[[[[2.95303911e-01 8.87675881e-02 1.15745710e-02 ... 0.00000000e+00
    0.00000000e+00 0.00000000e+00]
   [4.36791569e-01 1.14750065e-01 4.10810597e-02 ... 0.00000000e+00
    0.00000000e+00 0.00000000e+00]
   [2.16136679e-01 8.16718042e-02 5.30701168e-02 ... 0.00000000e+00
    0.00000000e+00 0.00000000e+00]
   ...
   [2.78009102e-04 4.33045533e-03 5.25094569e-02 ... 0.00000000e+00
    0.00000000e+00 0.00000000e+00]
   [2.50567198e-01 7.70653188e-02 5.28139733e-02 ... 0.00000000e+00
    0.00000000e+00 0.00000000e+00]
   [6.79462478e-02 6.82771876e-02 6.81128129e-02 ... 0.00000000e+00
    0.00000000e+00 0.00000000e+00]]]


 [[[1.27067053e+00 1.18729367e-03 2.92891893e-03 ... 0.00000000e+00
    0.00000000e+00 0.00000000e+00]
   [1.99715662e+00 1.59483515e-02 3.15287593e-03 ... 0.00000000e+00
    0.00000000e+00 0.00000000e+00]
   [9.76274069e-03 4.11544852e-06 2.69944053e-02 ... 0.00000000e+00
    0.00000000e+00 0.00000000e+00]
   ...
   [2.78178000e+00 1.09189689e-01 3.67725790e-02 ... 0.

Get top k attended words for each head, for each example in batch, for each layer

In [ ]:
def find_top_attention(scores_mat, axis, k):
  indices = scores_mat.argsort(axis=axis)[:, :, :, :-(k+1):-1]
  vals = np.take_along_axis(scores_mat, indices, axis=axis)
  return indices, vals

In [ ]:
all_tokens = tokenizer.convert_ids_to_tokens(testexam["input_ids"])

We want the position (index) of the token, the attention value, and the actual token itself.

In [ ]:
indexes, values = find_top_attention(attention_matrix_importance, 3, 10)

In [ ]:
print(indexes.shape)

(12, 1, 12, 10)


In [ ]:
def get_tokens(index_matrix):
  highest_tokens = []
  for i in range(indexes.shape[0]):
    row_tokens = []
    for j in range(indexes.shape[1]):
      batch_tokens = []
      for k in range(indexes.shape[2]):
        tokens = [all_tokens[idx] for idx in indexes[i][j][k]]
        batch_tokens.append(tokens)
      row_tokens.append(batch_tokens)
    highest_tokens.append(row_tokens)
  return np.array(highest_tokens)

highest_tokens = get_tokens(indexes)
print(highest_tokens.shape)

(12, 1, 12, 10)


Get the attention of a token at a position for each layer and head. Take one example at a time as each example has different tokens. Pros: can isolate for layers and/or heads. Cons: not much context for the attention scores

In [ ]:
def position_attention(agg_matrix, example, position):
  new_mat = agg_matrix[:, example, :]
  new_mat = new_mat.squeeze()
  print(new_mat.shape)
  dataframe=[]
  for i in range(new_mat.shape[0]):
    for j in range(new_mat.shape[1]):
      temp = new_mat[i,j].argsort()[::-1]
      temp = np.where(temp==position)[0].squeeze() + 1
      d = {"token":all_tokens[position], 'position':position, 
           'attention_scores':new_mat[i,j,position], 'layer':(i+1), 'head':(j+1),
           'rank':temp}
      dataframe.append(d)
  df = pd.DataFrame(dataframe)
  return df

In [ ]:
print(attention_matrix_importance[:,0].shape)

(12, 12, 2048)


In [ ]:
position_df = position_attention(attention_matrix_importance, 0, 0)
position_df[position_df["head"]==1]

(12, 12, 2048)


,token,position,attention_scores,layer,head,rank
0,<s>,0,0.295304,1,1,1
12,<s>,0,1.270671,2,1,1
24,<s>,0,0.558289,3,1,1
36,<s>,0,2.003426,4,1,1
48,<s>,0,0.072135,5,1,2
60,<s>,0,0.181244,6,1,8
72,<s>,0,0.019496,7,1,94
84,<s>,0,0.049842,8,1,1
96,<s>,0,0.000346,9,1,15
108,<s>,0,0.000042,10,1,90


If really needed, can just have the full matrix of the position, ranks, attention scores, layers, and heads of each token per example. 

Tokens are all the tokens in the example. Position is the location of the token with zero-based indexing. Attention scores are the aggregate, scaled attention scores.

Layer goes from 1 to 12.
Head goes from 1 to 12.

Rank is the attention score rank with respect to layer and head. Goes from 1 to number of tokens in the example

Pros: can search up whatever is needed. Has access to all the information and can be extracted for comparisons Cons: have to know what you want and manually look it up

In [ ]:
def full_matrix(agg_matrix, example):
  new_mat = agg_matrix[:, example]
  new_mat = new_mat.squeeze()
  print(new_mat.shape)
  dataframe=[]
  for i in range(new_mat.shape[0]):
    for j in range(new_mat.shape[1]):
      temp = new_mat[i,j].argsort()[::-1]      
      for k in range(new_mat.shape[2]):
        temp2 = np.where(temp==k)[0].squeeze() + 1
        d = {"token":all_tokens[k], 'position':k, 
            'attention_scores':new_mat[i,j,k], 'layer':(i+1), 'head':(j+1),
            'rank':temp2}
        dataframe.append(d)
  df = pd.DataFrame(dataframe)
  return df

In [ ]:
full_mat = full_matrix(attention_matrix_importance,0)

(12, 12, 2048)


In [ ]:
full_mat[(full_mat['head']==1) & (full_mat['layer']==1)]

,token,position,attention_scores,layer,head,rank
0,<s>,0,0.295304,1,1,1
1,<,1,0.088768,1,1,19
2,p,2,0.011575,1,1,429
3,>,3,0.074105,1,1,39
4,Episode,4,0.041921,1,1,195
...,...,...,...,...,...,...
2043,<pad>,2043,0.000000,1,1,1373
2044,<pad>,2044,0.000000,1,1,1374
2045,<pad>,2045,0.000000,1,1,1375
2046,<pad>,2046,0.000000,1,1,756
